<a href="https://colab.research.google.com/github/Terrencebosco/Build_week_2/blob/master/Build_week_2_(scrape).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# imports
import requests
from bs4 import BeautifulSoup
from csv import writer ###
import matplotlib.pyplot as plt
import seaborn as sns
from time import sleep
from random import randint
import pandas as pd
import numpy as np
import re ###

In [ ]:
# pulls uniqe url from over all listing page for each listing
pages = np.arange(1,51)
values = []

for page in pages:
    
    data = requests.get('https://www.cars.com/for-sale/searchresults.action/?page='+str(page)+'&perPage=100&prMx=10000&rd=250&searchSource=PAGINATION&sort=relevance&stkTypId=28881&zc=33408')
    soup = BeautifulSoup(data.content, 'html.parser')
    search = soup.findAll('div', class_='listing-row__badges')
    
    sleep(randint(2,3))
    
    for post in search:
        values.append(post['data-submit-badge'])

len(values)

In [ ]:
# remove unwated text (formatting when pulling data)
srip = 'Based on EPA mileage ratings. Use for comparison purposes only. Actual mileage will vary depending on driving conditions, driving habits, vehicle maintenance, and other factors.'

In [ ]:
# pulling info from each web listing for each car

car_names = []
car_mileage = []
price = []
ext_color = []
int_color = []
city_mpg = []
high_mpg = []
drivetrain = []
transmission = []
engine = []

for value in values:
    url = f'https://www.cars.com/vehicledetail/detail/{value}/overview/'
    data = requests.get(url)
    soup = BeautifulSoup(data.content, 'html.parser')
    sleep(randint(2,5))
    
    # car name
    try:
        car_name = soup.find(class_='vehicle-info__title-container').h1.find_next('h1').get_text()
    except:
        'missing'
    car_names.append(car_name)
    
    # mileage
    try:
        mileage = soup.find(class_="vdp-cap-price__mileage--mobile vehicle-info__mileage mileage_margin").get_text()
    except:
        'missing'
    car_mileage.append(mileage)

    # price
    try:
        prices = soup.find('span', class_="vehicle-info__price-display vehicle-info__price-display--dealer cui-heading-2").text
    except:
        'missing'
    price.append(prices)

    # ext color 
    try:
        car_color = soup.findAll(class_="vdp-details-basics__item")[1].get_text().strip('\n').strip(' ')
    except:
        'missing'
    ext_color.append(car_color)
    
    # int color
    try:
        inside = soup.findAll(class_="vdp-details-basics__item")[3].get_text().strip('\n') 
    except:
        'missing'
    int_color.append(inside)
    
    # city mpg
    try:
        c_mpg = soup.find_all(class_="vdp-details-basics__item")[2].get_text().strip('\n').strip(srip).strip('City MPG:')
    except:
        'missing'
    city_mpg.append(c_mpg)
    
    # highway mpg
    try:
        h_mpg = soup.find_all(class_="vdp-details-basics__item")[4].get_text().strip('\n').strip(srip).strip('Highway MPG: ')
    except:
        'missing'
    high_mpg.append(h_mpg)
    
    # drivetrain
    try:
        drive = soup.findAll(class_="vdp-details-basics__item")[6].get_text().strip('\n').strip('Drivetrain:').strip(' ')
    except:
        'missing'
    drivetrain.append(drive)
    
    # transmission
    try:
        trans = soup.findAll(class_="vdp-details-basics__item")[7].get_text().strip('\n').strip('Transmission: ')
    except:
        'missing'
    transmission.append(trans)
    
    # engine
    try:
        eng = soup.findAll(class_="vdp-details-basics__item")[8].get_text().strip('\n').strip('Engine:').strip(' ')
    except:
        'missing'
    engine.append(eng)

In [ ]:
# turn appended lists into a dataframe
df = pd.DataFrame(list(zip(car_names, car_mileage, price, ext_color,
                           int_color, city_mpg, high_mpg, transmission, drivetrain, engine)), 
               columns =['car', 'mileage', 'price', 'ext_color', 'int_color',
                        'city_mpg', 'high_mpg', 'transmission', 'drivetrain', 'engine'])
df = df[~df['high_mpg'].str.contains('Stock')]